Review first few rows of CSV data

In [1]:
import pandas as pd

# Load the provided CSV file
file_path = '/workspaces/code/services/typedb/sp800-53b-control-baselines.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to understand its structure
df.head(10)


,sp800_53b_ctrl_id,sp800_53b_ctrl_domain,sp800_53b_ctrl_title,sp800_53b_ctrl_withdrawn,sp800_53b_baseline_privacy,sp800_53b_security_baseline_low,sp800_53b_security_baseline_moderate,sp800_53b_security_baseline_high
0,AC-1,Access Control,Policy and Procedures,NaN,x,x,x,x
1,AC-2,Access Control,Account Management,NaN,NaN,x,x,x
2,AC-2(1),Access Control,Account Management | Automated System Account ...,NaN,NaN,NaN,x,x
3,AC-2(2),Access Control,Account Management | Automated Temporary and E...,NaN,NaN,NaN,x,x
4,AC-2(3),Access Control,Account Management | Disable Accounts,NaN,NaN,NaN,x,x
5,AC-2(4),Access Control,Account Management | Automated Audit Actions,NaN,NaN,NaN,x,x
6,AC-2(5),Access Control,Account Management | Inactivity Logout,NaN,NaN,NaN,x,x
7,AC-2(6),Access Control,Account Management | Dynamic Privilege Management,NaN,NaN,NaN,NaN,NaN
8,AC-2(7),Access Control,Account Management | Privileged User Accounts,NaN,NaN,NaN,NaN,NaN
9,AC-2(8),Access Control,Account Management | Dynamic Account Management,NaN,NaN,NaN,NaN,NaN


Extend the schema to support the structure

In [12]:
from typedb.driver import TypeDB, SessionType, TransactionType

def extend_schema_with_sp800_53b_baselines():
    with TypeDB.core_driver("localhost:1729") as driver:
        with driver.session("grc_matrix", SessionType.SCHEMA) as session:
            with session.transaction(TransactionType.WRITE) as transaction:
                schema_extension_query = '''
                define
                sp800_53b_ctrl_baselines sub NISTFramework,
                    owns sp800_53b_ctrl_id,
                    owns sp800_53b_ctrl_domain,
                    owns sp800_53b_ctrl_title,
                    owns sp800_53b_ctrl_withdrawn,
                    owns sp800_53b_baseline_privacy,
                    owns sp800_53b_security_baseline_low,
                    owns sp800_53b_security_baseline_moderate,
                    owns sp800_53b_security_baseline_high;

                sp800_53b_ctrl_id sub attribute, value string;
                sp800_53b_ctrl_domain sub attribute, value string;
                sp800_53b_ctrl_title sub attribute, value string;
                sp800_53b_ctrl_withdrawn sub attribute, value string;
                sp800_53b_baseline_privacy sub attribute, value string;
                sp800_53b_security_baseline_low sub attribute, value string;
                sp800_53b_security_baseline_moderate sub attribute, value string;
                sp800_53b_security_baseline_high sub attribute, value string;

                has_control sub relation,
                    relates control,
                    relates domain;

                has_name sub relation,
                    relates control,
                    relates name;

                has_domain sub relation,
                    relates control,
                    relates domain;
                '''
                transaction.query.define(schema_extension_query)
                transaction.commit()
                print("Schema extended with sp800_53b_ctrl_baselines entity successfully.")

extend_schema_with_sp800_53b_baselines()


Schema extended with sp800_53b_ctrl_baselines entity successfully.


Insert the CSV data.

In [13]:
import pandas as pd
from typedb.driver import TypeDB, SessionType, TransactionType

# Load CSV data
file_path = '/workspaces/code/services/typedb/sp800-53b-control-baselines.csv'
df = pd.read_csv(file_path)

def insert_sp800_53b_controls_from_csv():
    with TypeDB.core_driver("localhost:1729") as driver:
        with driver.session("grc_matrix", SessionType.DATA) as session:
            with session.transaction(TransactionType.WRITE) as transaction:
                for index, row in df.iterrows():
                    control_query = f'''
                    insert
                        $control isa sp800_53b_ctrl_baselines,
                            has sp800_53b_ctrl_id "{row['sp800_53b_ctrl_id']}",
                            has sp800_53b_ctrl_domain "{row['sp800_53b_ctrl_domain']}",
                            has sp800_53b_ctrl_title "{row['sp800_53b_ctrl_title']}",
                            has sp800_53b_ctrl_withdrawn "{row['sp800_53b_ctrl_withdrawn']}",
                            has sp800_53b_baseline_privacy "{row['sp800_53b_baseline_privacy']}",
                            has sp800_53b_security_baseline_low "{row['sp800_53b_security_baseline_low']}",
                            has sp800_53b_security_baseline_moderate "{row['sp800_53b_security_baseline_moderate']}",
                            has sp800_53b_security_baseline_high "{row['sp800_53b_security_baseline_high']}";
                    '''
                    transaction.query.insert(control_query)
                transaction.commit()
                print("Data inserted successfully.")

insert_sp800_53b_controls_from_csv()


Data inserted successfully.
